In [1]:
import numpy as np
import sympy as sp

In [2]:
(x, y, z, v, w) = sp.symbols("x, y, z, v, w")


# def f2(x, y, z, v, w):
#     return (
#         2 * (v - 3 * z) ** 3
#         + 15 * w**2
#         + 0.3 * x**2
#         + 2 * y**2
#         - 6 * v**3
#         - 2 * z**2
#         + 0.5 * w**2
#         + 18 * w
#     )
    
# def f2 (x, y): 
#     return 3* x**2 + 2 * y **2 + x + 10

# def ev_point_gradient(point, gradient):
#     return [g.subs(point) for g in gradient]

# def ev_point_hessian(point, hessian):
#     return hessian.subs(point)

# def newton_method(objective_func, initial_point, tol=1e-10, max_iter=1000):
#     variables = list(initial_point.keys())
#     expression = objective_func(*variables)
#     gradient = [sp.diff(expression, var) for var in variables]
#     hessian = sp.hessian(expression, variables)

#     previous_value = float('inf')
#     current_point = initial_point.copy()

#     for _ in range(1, max_iter):
#         current_value = float(expression.subs(current_point).evalf())
        
#         if abs(previous_value - current_value) < tol:
#             print(f"Convergencia: valor de la función = {current_value:.4f}")
#             print(f"Punto: {current_point}")
#             break

#         previous_value = current_value
#         gradient_values = np.array(ev_point_gradient(current_point, gradient))
#         hessian_values = np.array(ev_point_hessian(current_point, hessian))

#         if np.linalg.det(hessian_values) == 0:
#             print("La matriz Hessiana no es invertible.")
#             break

#         hessian_inverse = np.linalg.inv(hessian_values)
#         delta = hessian_inverse @ gradient_values

#         for j, key in enumerate(current_point.keys()):
#             current_point[key] -= delta[j]

#     return current_point, current_value

# initial_point = {x: 0, y: 1}
# optimal_point, optimal_value = newton_method(f2, initial_point)
# print(f"Punto óptimo: {optimal_point}")
# print(f"Valor óptimo de la función: {optimal_value:.4f}")


In [3]:
variables = (x, y)
tol = 0.01
max_iter = 100
# LA CANTIDAD DE VARIABLES SIMBOLICAS Y VALORES DEBEN SER LOS MISMOS    
nofrand = 10
initial_bounds = (-10000000, 10000000)

f = lambda x, y: 3 * x**2 + 2 * y**2 + x + 10

# El ultimo parametro indica cuantos valores se quieren generar en el array, es mejor que declarar las variables una por una
random_point = np.random.uniform(*initial_bounds, 2)

expresion = f(*variables)
grad = [sp.diff(expresion, var) for var in variables]
H = sp.hessian(expresion, variables)

previous_value = 1e9
count = 0
for i in range(max_iter):

    count += 1

    J = [g.subs(zip(variables, random_point)) for g in grad]
    H_eval = H.subs(zip(variables, random_point))

    current_value = f(*random_point)
    if abs(previous_value - current_value) < tol:
        print(f"Finished: {current_value}")
        print(f"Iteracion: {count}\nPoint {random_point}")
        break
    previous_value = current_value

    H_eval = H.subs(zip(variables, random_point))
    H_eval = np.array(H_eval, dtype="float")  # Convierte la expresión en números
    H_inv = np.linalg.inv(H_eval)
    new_point = random_point - H_inv @ J  # producto escalar

    random_point = new_point

Finished: 9.91666666666667
Iteracion: 3
Point [-0.166666666666667 0]
